In [1]:
import os
import re
import json
import faiss
import warnings
import huggingface_hub

from openai import OpenAI

from langchain.schema import Document
from langchain_community.vectorstores.faiss import FAISS

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker


os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /data/ephemeral/home/.cache/huggingface/token
Login successful


In [2]:
client = OpenAI()
model = "gpt-4o"

# client = OpenAI(
#     api_key=upstage_api_key,
#     base_url="https://api.upstage.ai/v1/solar"
# )
# model = "solar-pro"

In [3]:
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    
    return cleaned_response

def query_expansion(query, model:str, client:OpenAI):
    content = (
        """
        당신은 한국어 질의를 이해하여 정확한 문서들을 검색하도록 질의를 개선하는 전문가입니다.
        사용자의 질문 의도를 명확히 파악하고, 그 의도를 유지하면서 검색 시스템이 관련 문서를 더 잘 찾을 수 있도록 질의를 더 명확하게 만드세요.
        다의어(여러 가지 의미를 가질 수 있는 단어)를 포함하는 경우, 주어진 문맥에서 가장 적절한 의미를 선택하여 확장하세요.
        질문의 키워드가 여러 분야에 걸쳐 사용될 수 있는 경우, 사용자가 의도한 분야 또는 가장 가능성이 높은 분야를 명확히 하세요.
        질문의 의도가 불명확하거나 모호할 경우, 일반적인 해석을 추가하여 검색의 정확도를 높이세요.

        다음은 몇 가지 예시입니다.

        예제1
            입력: Thomas Alva Edison은 누구인가요?
            출력: 특정 인물에 대해 묻는 질문입니다. Thomas Alva Edison(토머스 앨바 에디슨)라는 인물은 누구인가요?

        예제2
            입력: 온난 전선이 발생하면 이후 날씨는 어떻게 되나?
            출력: 기후에서 발생하는 온난 전선이 발생한 후의 날씨에 대한 질문입니다. 온난 전선이 발생하면 이후의 기상 현상에 대해 설명해주세요.

        예제3
            입력: 짚신 벌레의 번식은 어떻게 이루어지나?
            출력: 짚신 벌레의 번식 과정에 대해 묻는 질문입니다. 짚신 벌레는 어떻게 번식하며, 그 과정에서 어떤 특징이 나타나는지 설명해주세요.

        예제4
            입력: 작은 기체 하나의 질량을 어떻게 구할 수 있어?
            출력: 물리학에서 작은 기체의 질량을 구하는 방법에 대한 질문입니다. 기체의 질량을 구하는 방법을 밀도와 온도를 기반으로 설명해주세요.

        예제5
            입력: 곤충의 생태를 관찰할 때 사용할 수 있는 방법은?
            출력: 곤충의 생태를 관찰하는 방법에 대한 질문입니다. 곤충을 관찰하기 위한 도구와 방법, 그리고 생태계에서 관찰할 수 있는 곤충의 행동이나 특징을 설명해주세요.

        예제6
            입력: 난관의 기능에 대해 알려줘.
            출력: 여성 생식기에서 난관의 기능에 대해 묻는 질문입니다. 난관은 어떤 역할을 하며, 생식 과정에서 어떻게 기여하는지 설명해주세요.

        확장된 질의는 자연스러운 문장 형태로 제공되어야 합니다.
        반환하는 형식은 반드시 JSON 포맷이어야 하며, 모든 문자열은 쌍따옴표로 감싸야 합니다.
        { "query": "확장된 자연스러운 질의" }.'
        """
    )


    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role" : "system", "content" : content},
            {"role" : "user", "content" : query}
        ],
    )
    
    response = completion.choices[0].message.content
    response = clean_json_response(response)
    
    try:
        json_response = json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response


In [4]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

def load_query(query_path, model, client, output_path):
    queries = load_jsonl(query_path)
    expanded_queries = []

    for query_data in queries:
        eval_id = query_data['eval_id']
        msg_history = query_data['msg']
        
        conversation_history = " ".join([msg['content'] for msg in msg_history if msg['role'] == 'user'])
        
        expanded_query_result = query_expansion(conversation_history, model, client)
        print(eval_id)
        print(conversation_history)
        print(expanded_query_result['query'], '\n')
        
        expanded_queries.append({
            "eval_id": eval_id,
            "original_conversation": conversation_history,
            "expanded_query": expanded_query_result.get('query', 'Expansion failed')
        })
    
    # Save expanded queries to a jsonl file
    with open(output_path, 'w', encoding='utf-8') as f:
        for query in expanded_queries:
            json.dump(query, f, ensure_ascii=False)
            f.write('\n')

In [5]:
load_query("../dataset/eval.jsonl", model, client, "../dataset/gpto1-expanded_query.jsonl")

78
나무의 분류에 대해 조사해 보기 위한 방법은?
나무의 분류 방법을 조사하기 위한 구체적인 방법에 대해 묻는 질문입니다. 나무를 분류하는 데 사용할 수 있는 생물학적 기준과 분류 체계를 설명하고, 이를 조사하기 위한 연구 방법이나 자료가 있는지 알려주세요. 

213
각 나라에서의 공교육 지출 현황에 대해 알려줘.
세계 각 나라에서 공교육에 대한 지출 현황에 대해 알려주세요. 각국의 공교육 예산 및 교육 부문에 대한 투자가 어떻게 이루어지고 있는지 설명해주세요. 

107
기억 상실증 걸리면 너무 무섭겠다. 어떤 원인 때문에 발생하는지 궁금해.
기억 상실증의 원인에 대해 알고 싶습니다. 기억 상실증은 주로 어떤 이유로 발생하며, 그 원인에는 어떤 것들이 있는지 설명해주세요. 

81
통학 버스의 가치에 대해 말해줘.
학생들의 이동 수단으로 사용되는 통학 버스의 가치와 중요성은 무엇인가요? 

280
Dmitri Ivanovsky가 누구야?
특정 과학자에 대해 묻는 질문입니다. Dmitri Ivanovsky(드미트리 이바노프스키)는 누구이며, 그의 주요 업적은 무엇인가요? 

10
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?
피임 방법 중 하나로 약을 사용하는 방법의 효과와 안전성에 대해 설명해주세요. 약물 피임법은 실용적이고 효율적인 방법인지 논의해주세요. 

100
헬륨이 다른 원소들과 반응을 잘 안하는 이유는?
헬륨 원소가 다른 원소들과 화학 반응을 잘 일으키지 않는 이유는 무엇인가요? 헬륨의 화학적 특성과 관련된 설명을 부탁드립니다. 

279
문맹 비율이 사회 발전에 미치는 영향은?
문맹 비율이 사회의 경제적, 문화적, 교육적 발전에 미치는 영향에 대해 설명해주세요. 문맹률이 높은 사회에서 발생할 수 있는 문제점과 이를 개선하기 위한 방안도 포함해주세요. 

42
이란 콘트라 사건이 뭐야 이 사건이 미국 정치에 미친 영향은?
이란 콘트라 사건이란 무엇이며, 이 사건이 미국 정치에 미친 영향은 무엇인가요? 

308
자기장이 얼마나 센지